# SpaceX Launch Records Dashboard

This notebook contains the code for a Plotly Dash application to analyze SpaceX launch data, along with the answers to the required questions.

## Setup
The following code sets up the Dash application with a dropdown, pie chart, range slider, and scatter plot.

In [ ]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the SpaceX data into a pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# TASK 1: Add a dropdown list to enable Launch Site selection
# Get unique launch sites for dropdown options
launch_sites = [{'label': 'All Sites', 'value': 'ALL'}]
launch_sites.extend([{'label': site, 'value': site} for site in spacex_df['Launch Site'].unique()])

# Create app layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
    
    # Dropdown for selecting launch site
    dcc.Dropdown(id='site-dropdown',
                 options=launch_sites,
                 value='ALL',
                 placeholder='Select a Launch Site here',
                 searchable=True
                ),
    html.Br(),

    # TASK 2: Add a pie chart to show the total successful launches
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    # TASK 3: Add a slider to select payload range
    html.P("Payload range (Kg):"),
    dcc.RangeSlider(id='payload-slider',
                    min=0,
                    max=10000,
                    step=1000,
                    marks={i: str(i) for i in range(0, 10001, 1000)},
                    value=[min_payload, max_payload]
                   ),
    html.Br(),

    # TASK 4: Add a scatter chart to show the correlation between payload and success
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

# TASK 2: Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        # For "All Sites", count successful launches (class=1) per site
        success_counts = spacex_df[spacex_df['class'] == 1].groupby('Launch Site').size().reset_index(name='count')
        fig = px.pie(success_counts, 
                     values='count', 
                     names='Launch Site', 
                     title='Total Success Launches by Site')
        return fig
    else:
        # For a specific site, count occurrences of class=0 and class=1
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        class_counts = filtered_df['class'].value_counts().reset_index()
        class_counts.columns = ['class', 'count']
        fig = px.pie(class_counts, 
                     values='count', 
                     names='class', 
                     title=f'Success vs. Failed Launches for {entered_site}')
        return fig

# TASK 4: Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def get_scatter_chart(entered_site, payload_range):
    filtered_df = spacex_df[
        (spacex_df['Payload Mass (kg)'] >= payload_range[0]) & 
        (spacex_df['Payload Mass (kg)'] <= payload_range[1])
    ]
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, 
                         x='Payload Mass (kg)', 
                         y='class', 
                         color='Booster Version Category',
                         title='Payload vs. Success for All Sites')
    else:
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        fig = px.scatter(filtered_df,
                         x='Payload Mass (kg)', 
                         y='class', 
                         color='Booster Version Category',
                         title=f'Payload vs. Success for {entered_site}')
    return fig

# Run the app
if __name__ == '__main__':
    app.run(debug=True, port=8052)

## Answers to Questions

Using the dashboard, the following insights were obtained:

1. **Which site has the largest successful launches?**  
   KSC LC-39A (around 12 successful launches).

2. **Which site has the highest launch success rate?**  
   KSC LC-39A (approximately 92.3% success rate).

3. **Which payload range(s) has the highest launch success rate?**  
   2000-4000 kg and 4000-6000 kg (around 83.3% success rate).

4. **Which payload range(s) has the lowest launch success rate?**  
   0-1000 kg (around 40% success rate).

5. **Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?**  
   B5 (typically 100% success rate).